# Algorithm

### a. Download the files from PBI Server
1. Download all files
2. Place them in the selected folder
3. Place the Jupyter notebook into the same folder
### b. Process the .pbix files - steps for each file
##### Preparation - create the temp directory for original archives contents
1. Unarchive the pbix file into the folder with the same name, which is located in the `temp` directory
2. Go to PBIX folder
3. Remove the `SecurityBindings` file
4. Remove SecurityBindings recors from the `[Content_Types].xml` (for instance, replace the file with the prepared one)
5. Modify the `Report/Layout` file - replace the values
6. Create the new archive using the PBIX folder contents
##### Remove the folder with Layouts
### c. Check the files and publish them to PBI server
1. Open each file
2. Save and publish

# Algorithm implementation

In [2]:
import os
import re
import calendar
import datetime
import shutil
import json

from zipfile import ZipFile

#### Calculating the date values

In [3]:
CURRENT_DATE = datetime.date.today()
PREVIOUS_MONTH = CURRENT_DATE - datetime.timedelta(days=CURRENT_DATE.day)

NEW_VALUE_YEAR = PREVIOUS_MONTH.year
NEW_VALUE_MONTH = PREVIOUS_MONTH.month
NEW_VALUE_MONTH_ABBR = calendar.month_abbr[NEW_VALUE_MONTH]
NEW_VALUE_QUARTER = (NEW_VALUE_MONTH + 2) // 3

In [4]:
print(CURRENT_DATE, PREVIOUS_MONTH)
print(NEW_VALUE_YEAR, NEW_VALUE_MONTH, NEW_VALUE_MONTH_ABBR, NEW_VALUE_QUARTER)

2023-03-15 2023-02-28
2023 2 Feb 1


#### Organizing the workspace. Creating the temporary and results folders

In [5]:
"""Path constants"""
WORKSPACE_PATH = os.getcwd()  # workspace directory with all the data
SOURCE_PATH = os.path.join(WORKSPACE_PATH, "source")  # directory with original .pbix files
TEMP_PATH = os.path.join(WORKSPACE_PATH, "temp")      # directory for temporary files
RESULTS_PATH = os.path.join(WORKSPACE_PATH, "result") # directory for the modified .pbix files

In [6]:
def create_workspace_hierarchy():
    """Checking if directories exist"""
    if not os.path.exists(RESULTS_PATH):
        os.mkdir(RESULTS_PATH)
    if not os.path.exists(TEMP_PATH):
        os.mkdir(TEMP_PATH)
    if not os.path.exists(SOURCE_PATH):
        os.mkdir(SOURCE_PATH)
        print("Please, place all the .pbix files into the 'source' directory, and then rerun the script")
    else:
        print("The directories structure is OK")

In [7]:
create_workspace_hierarchy()

Please, place all the .pbix files into the 'source' directory, and then rerun the script


#### Iterating through files

1. Get the list of files in the `source` directory

In [83]:
source_pbix_files_names = [file_name[:-5] for file_name in os.listdir(SOURCE_PATH) if file_name[-5:] == ".pbix"]
print(source_pbix_files_names)

['Asia Regional Dashboard-GCL v3', 'test']


In [84]:
# Getting first element for algorithm example
pbix_file_name = source_pbix_files_names[0]
pbix_file_name

'Asia Regional Dashboard-GCL v3'

2. Unzip the .pbix file to the `temp` directory

In [104]:
# paths for the original .pbix file (archive) and temporary folder, to which the file will be unarchived
source_zip_path = os.path.join(SOURCE_PATH, pbix_file_name + ".pbix")
temp_dir_path = os.path.join(TEMP_PATH, pbix_file_name)

# opening the zip file in READ mode
with ZipFile(source_zip_path, 'r') as source_zip_file:
    print('Extracting all the files from', source_zip_path, "to", temp_dir_path)
    source_zip_file.extractall(path=temp_dir_path)

Extracting all the files from c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\source\Asia Regional Dashboard-GCL v3.pbix to c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\Asia Regional Dashboard-GCL v3


3. Remove the `SecurityBindings` file

In [86]:
security_bindings_file_path = os.path.join(temp_dir_path, "SecurityBindings")
if os.path.exists(security_bindings_file_path):
    os.remove(security_bindings_file_path)

4. Remove SecurityBindings recors from the `[Content_Types].xml` (for instance, replace the file with the prepared one)

In [87]:
content_types_file_path = os.path.join(temp_dir_path, "[Content_Types].xml")

with open(content_types_file_path, "r") as content_types_file:
    xml = content_types_file.read()

updated_xml = xml.replace('<Override PartName="/SecurityBindings" ContentType="" />', "")
with open(content_types_file_path, "w") as content_types_file:
    content_types_file.write(updated_xml)

##### 5. Update the `Report/Layout` file

1. Month and year, to which the file should be updated

2023 3 Mar 1


2. List of pattern and replacements pairs

In [8]:
PERIOD_REPLACEMENTS = [
    # pattern, new value
    ('(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)', current_month_abbr),  # month eng
    ('\d{1,2}月', str(current_month) + "月"),   # month chn
    ('Q [1-4]', 'Q ' + str(current_quarter)),   # quarter eng (with space)
    ('Q[1-4]', 'Q' + str(current_quarter)),     # quarter eng (without space)
    #('\d{1,2}月', str(current_month) + "月"),  # quarter chn
    (str(current_year - 1), str(current_year))  # year
]

NameError: name 'current_month_abbr' is not defined

In [72]:
for (pattern, new_value) in PERIOD_REPLACEMENTS:
    print("new value - {}\t\tpattern - {}".format(new_value, pattern))

new value - Mar		pattern - (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)
new value - 3月		pattern - \d{1,2}月
new value - Q 1		pattern - Q [1-4]
new value - Q1		pattern - Q[1-4]
new value - 2023		pattern - 2022


3. Reading the `Layout` file

In [73]:
layout_file_path = os.path.join(temp_dir_path, "Report", "Layout")

with open(layout_file_path, "r", encoding="utf-16-le") as layout_file:
    layout_data = layout_file.read()

4. Functions for replacement

In [40]:
def replace_period(text:str, period_pattern:str, period_new_value:str):
    search_pattern = create_value_expression(period_pattern)
    new_value = create_value_expression(period_new_value)
    return re.sub(search_pattern, new_value, text)


def create_value_expression(value:str):
    return '"Value": "\'{}\'"'.format(value)

5. Replacing the patterns in the `Layout` file

In [78]:
for (pattern, new_value) in PERIOD_REPLACEMENTS:
    layout_data = replace_period(layout_data, pattern, new_value)

In [65]:
print(re.findall(r'"Value": "\'2023\'"', layout_data))

[]


##### 6. Assemble the new .pbix file

In [49]:
result_pbix_file_path = os.path.join(RESULTS_PATH, pbix_file_name + ".pbix")
                                     
with ZipFile(result_pbix_file_path, mode="w") as result_pbix_archive:
    for folderName, subfolders, filenames in os.walk(temp_dir_path):
        for filename in filenames:
            # create complete filepath of file in directory
            temp_dir_file_path = os.path.join(folderName, filename)
            # path for the archive hierarchy
            archive_path = temp_dir_file_path[len(temp_dir_path)+1:]
            # selecting the compression mode depending on archive file type
            file_compression_mode = result_pbix_archive.ZIP_STORED if filename == "DataModel" else result_pbix_archive.ZIP_DEFLATED
            # writing the file to result archive
            result_pbix_archive.write(temp_dir_file_path, arcname=archive_path, compress_type=file_compression_mode)

In [8]:
for directory_path, _, filenames in os.walk(WORKSPACE_PATH):
    for filename in filenames:
        if filename[-5:] == ".pbix" and RESULTS_PATH not in directory_path and TEMP_PATH not in directory_path:
            subdirectory_path = directory_path[len(WORKSPACE_PATH) + 1:]
            print(directory_path, filename, subfolders, sep="    ")


c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\ws1    Asia Regional Dashboard-GCL v3.pbix    []
c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\ws1    test.pbix    []
c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\ws2    Asia Regional Dashboard-GCL v3.pbix    []
c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\ws2    test.pbix    []


In [19]:
subdirectory = "ws1"
filename = "test.pbix"
result_pbix_file_path = os.path.join(RESULTS_PATH, subdirectory, filename)
temporary_files_path = os.path.join(TEMP_PATH, subdirectory, filename[:-5])
print(result_pbix_file_path)
print(os.path.join(temporary_files_path))



c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\result\ws1\test.pbix
c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test


In [30]:
import zipfile

with ZipFile(result_pbix_file_path, mode="w") as result_pbix_archive:
    for directory_path, _, filenames in os.walk(temporary_files_path):
        print(directory_path, filenames)
        for filename in filenames:
            # create complete filepath of file in directory
            temp_dir_file_path = os.path.join(directory_path, filename)
            # path for the archive hierarchy
            archive_path = temp_dir_file_path[len(temporary_files_path)+1:]
            # selecting the compression mode depending on archive file type
            file_compression_mode = zipfile.ZIP_STORED if filename == "DataModel" else zipfile.ZIP_DEFLATED
            # writing the file to result archive
            print(temp_dir_file_path, archive_path, sep="\t")
            print()
            result_pbix_archive.write(temp_dir_file_path, arcname=archive_path, compress_type=file_compression_mode)

c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test ['DataModel', 'DiagramLayout', 'Metadata', 'Settings', 'Version', '[Content_Types].xml']
c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test\DataModel	DataModel

c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test\DiagramLayout	DiagramLayout

c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test\Metadata	Metadata

c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test\Settings	Settings

c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test\Version	Version

c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test\[Content_Types].xml	[Content_Types].xml

c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test\Report ['Layout']
c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test\Report\Layout	Report\Layout

c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\ws1\test\Report\StaticResources []
c:\Users\Kateryna_Nemc

In [25]:
def get_patterns_and_replacements(cli_arg_year, cli_arg_month):
    if not cli_arg_month and not cli_arg_year:
        current_date = datetime.date.today()
        period_for_new_values = current_date - datetime.timedelta(days=current_date.day)
    else:
        period_for_new_values = datetime.date(cli_arg_year, cli_arg_month, 1)
    
    previous_month = period_for_new_values - datetime.timedelta(days=period_for_new_values.day)

    NEW_VALUE_YEAR = period_for_new_values.year
    NEW_VALUE_MONTH = period_for_new_values.month
    NEW_VALUE_MONTH_ABBR = calendar.month_abbr[NEW_VALUE_MONTH]
    NEW_VALUE_QUARTER = (NEW_VALUE_MONTH + 2) // 3

    period_replacements = [
        # pattern, new value
        ("'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)'", NEW_VALUE_MONTH_ABBR),  # month eng
        ("'\d{1,2}月'", "'{}'月".format(NEW_VALUE_MONTH)),   # month chn
        ("'Q [1-4]'",   "'Q {}'".format(NEW_VALUE_QUARTER)),   # quarter eng (with space)
        ("'Q[1-4]'",    "'Q{}'".format(NEW_VALUE_QUARTER)),     # quarter eng (without space)
        ("'[1-4]季度'", "'{}季度'".format(NEW_VALUE_QUARTER)),  # quarter chn
        ("{}L".format(previous_month.year), "{}L".format(NEW_VALUE_YEAR))  # year
    ]
    return period_replacements

period_replacements = get_patterns_and_replacements(None, None)
period_replacements


[("'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)'", 'Feb'),
 ("'\\d{1,2}月'", "'2'月"),
 ("'Q [1-4]'", "'Q 1'"),
 ("'Q[1-4]'", "'Q1'"),
 ("'[1-4]季度'", "'1季度'"),
 ('2023L', '2023L')]

In [19]:
print("\nPATTERNS")
for period_pattern, period_new_value in period_replacements:
    print("{}  ->  {}".format(period_pattern, period_new_value))


PATTERNS
'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)'  ->  Feb
'\d{1,2}月'  ->  '2'月
'Q [1-4]'  ->  'Q 1'
'Q[1-4]'  ->  'Q1'
'[1-4]季度'  ->  '1季度'
2023L  ->  2023L


In [26]:
def replace_period(text, period_pattern, period_new_value):
    # The should be deffirent for search and replacement operations
    search_pattern = r'\\"Value\\":\\"{}\\"'.format(period_pattern)
    new_value = r'\\"Value\\":\\"{}\\"'.format(period_new_value)
    return re.sub(search_pattern, new_value, text)

In [28]:
layout_file_path = r"C:\\Users\\Kateryna_Nemchenko\\Desktop\\Bookmark Updates\\Layout"

def create_value_expression(value:str):
    return r'\\"Value\\":\\"{}\\"'.format(value)

with open(layout_file_path, "r", encoding="utf-16-le") as layout_file:
    layout_data = layout_file.read()

for pattern, new_value in period_replacements:
    search_pattern = r'\\"Value\\":\\"{}\\"'.format(period_pattern)
    new_value = '\\\\"Value\\\\":\\\\"{}\\\\"'.format(period_new_value)
    #search_pattern = create_value_expression(pattern)
    #new_value = create_value_expression(new_value)
    print(search_pattern, new_value, re.findall(search_pattern, layout_data), sep="\n")
    if re.findall(search_pattern, layout_data):
        fragment_index = re.finditer(search_pattern, layout_data).__next__().start()
        print(layout_data[fragment_index-100:fragment_index+100])
        new_layout_data = re.sub(search_pattern, new_value, layout_data)
        print(new_layout_data[fragment_index-100:fragment_index+100])
    print()


\\"Value\\":\\"2023L\\"
\"Value\":\"2023L\"
['\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\\":\\"2023L\\"', '\\"Value\

In [21]:
with open(layout_file_path, "r", encoding="utf-16-le") as layout_file:
    layout_data = layout_file.read()

updated_layout_data = layout_data
for pattern, new_value in period_replacements:
    initial_layout_data = updated_layout_data
    updated_layout_data = replace_period(initial_layout_data, pattern, new_value)
    print(initial_layout_data==updated_layout_data)

#with open(layout_file_path, "w", encoding="utf-16-le") as layout_file:
    #layout_file.write(layout_data)

False
False
True
True
True
True


In [15]:
def create_value_expression(value:str):
    return '\\\"Value\\\":\\\"\'{}\'\\\"'.format(value)

def replace_period(text:str, period_pattern:str, period_new_value:str):
    search_pattern = create_value_expression(period_pattern)
    new_value = create_value_expression(period_new_value)
    print(search_pattern, new_value, re.findall(search_pattern, text))
    return re.sub(search_pattern, new_value, text)

In [13]:
pbi_filename = "Asia Regional Dashboard-GCL v3.pbix"
temp_pbix_path = os.path.join(TEMP_PATH, subdirectory, pbi_filename[:-5])

layout_file_path = os.path.join(temp_pbix_path, "Report", "Layout")

print(temp_pbix_path)
print(layout_file_path)





NameError: name 'subdirectory' is not defined

In [67]:
test_text = r"ral\":{\"Value\":\"'Jan'\"}}]]}}}]"
test_text


'ral\\":{\\"Value\\":\\"\'Jan\'\\"}}]]}}}]'

# The script